<a href="https://colab.research.google.com/github/shreyanshchordia/covid19-global-analysis/blob/master/COVID_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Viewer is requested to view the [notebook](https://colab.research.google.com/gist/shreyanshchordia/13ea2b892ae1b5ecd6e21ab16d536762/covid-19.ipynb) on google colaboratory. This notebook outputs a bunch of plots and charts that cannot be displayed by github notebook viewer in static mode. Thank you.

#**Loading Libraries**

In [0]:
!pip install pycountry
import pandas as pd
import plotly.express as px
from datetime import datetime
import pycountry
import plotly.graph_objects as go
import numpy as np

#**Loading Datasets**

In [0]:
confirmed = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Corona/confirmed.csv")
death = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Corona/death.csv")
recovered = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Corona/recovered.csv")

In [0]:
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,4,4,5,7,7,7,11,16,21,22,22,22,24,24,40,40,74,84,94,110,110,120,170,174,237,273,281,299,349,367,423
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51,55,59,64,70,76,89,104,123,146,174,186,197,212,223,243,259,277,304,333,361,377,383
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,1,1,1,3,5,12,12,17,17,19,20,20,20,24,26,37,48,54,60,74,87,90,139,201,230,264,302,367,409,454,511,584,716,847,986,1171,1251,1320,1423,1468
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,39,39,53,75,88,113,133,164,188,224,267,308,334,370,376,390,428,439,466,501,525,545
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,3,3,3,4,4,5,7,7,7,8,8,8,10,14,16,17


In [0]:
death.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,2,4,4,4,4,4,4,4,6,6,7,7,11,14
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,2,2,2,2,2,4,5,5,6,8,10,10,11,15,15,16,17,20,20,21,22
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,4,4,4,7,9,11,15,17,17,19,21,25,26,29,31,35,44,58,86,105,130,152,173,193
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,3,3,3,6,8,12,14,15,16,17,18,21,22
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2


In [0]:
recovered.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,5,5,10,10,10,15,18,18
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,10,17,17,31,31,33,44,52,67,76,89,99,104,116,131
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,12,12,12,12,12,32,32,32,65,65,24,65,29,29,31,31,37,46,61,61,62,90,90,90,113
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,10,10,10,10,16,21,26,31,39
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,2,2,2,2


#**Analysis of Confirmation of Cases over time**

##**Building relevant dataframes**

In [0]:
top_10_confirmed = confirmed.sort_values(by=["4/7/20"],ascending=False,na_position='last')[0:10]
confirmed_flow = top_10_confirmed.melt(id_vars=["Province/State", "Country/Region", "Lat",'Long'],var_name="Date",value_name="Confirmed")
confirmed_flow.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,US,37.0902,-95.7129,1/22/20,1
1,NaN,Spain,40.0000,-4.0000,1/22/20,0
2,NaN,Italy,43.0000,12.0000,1/22/20,0
3,NaN,France,46.2276,2.2137,1/22/20,0
4,NaN,Germany,51.0000,9.0000,1/22/20,0


In [0]:
confirmed_melted = confirmed.melt(id_vars=["Province/State", "Country/Region", "Lat",'Long'],var_name="Date",value_name="Confirmed")
for i,string in enumerate(confirmed_melted['Date']):
    x = datetime.strptime(string,'%m/%d/%y')
    confirmed_melted['Date'][i] = x.strftime('%Y-%m-%d')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
confirmed_melted.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0
1,NaN,Albania,41.1533,20.1683,2020-01-22,0
2,NaN,Algeria,28.0339,1.6596,2020-01-22,0
3,NaN,Andorra,42.5063,1.5218,2020-01-22,0
4,NaN,Angola,-11.2027,17.8739,2020-01-22,0


In [0]:
cases = confirmed_melted
grp = cases.groupby(['Date', 'Country/Region'])['Confirmed'].max()
grp = grp.reset_index()
grp['Country'] =  grp['Country/Region']
grp.head()

,Date,Country/Region,Confirmed,Country
0,2020-01-22,Afghanistan,0,Afghanistan
1,2020-01-22,Albania,0,Albania
2,2020-01-22,Algeria,0,Algeria
3,2020-01-22,Andorra,0,Andorra
4,2020-01-22,Angola,0,Angola


In [0]:
group = confirmed.groupby(['Country/Region'])['4/7/20'].max()
group = group.reset_index()
group = group.sort_values(by=['4/7/20'])
group = group[group['4/7/20']>10000]
group.head()

,Country/Region,4/7/20
90,"Korea, South",10331
135,Portugal,12442
9,Austria,12639
23,Brazil,14034
120,Netherlands,19580


##**Plots**

In [0]:
fig = px.choropleth(grp, locations="Country", locationmode='country names', 
                     color="Confirmed", hover_name="Country/Region",hover_data = [grp.Confirmed],projection="mercator",
                     animation_frame="Date",width=1000, height=800,
                     color_continuous_scale='blues',
                     range_color=[100,20000],
                     title='World Map of Coronavirus')
fig.update_traces(reversescale=True)
fig.update(layout_coloraxis_showscale=True)
fig.show(renderer="colab")

In [0]:
fig = px.bar(group,y='4/7/20',x='Country/Region',color='4/7/20',title='Spread of corona virus in countries with more than 10,000 cases',
             color_continuous_scale='viridis',range_color=[10000,200000])
fig.update(layout_coloraxis_showscale=True)
fig.show(renderer='colab')

In [0]:
fig = px.pie(confirmed, values='4/7/20', names='Country/Region', color= '4/7/20',title="USA with most number of confirmed cases")
fig.update_traces(textposition='inside', textinfo='percent')
fig.show(renderer="colab")

In [0]:
fig = px.bar(top_10_confirmed,x="Country/Region",y="4/7/20",color="4/7/20",title="Top 10 countries with most number of confirmed cases",
color_continuous_scale='reds',range_color=[1000,150000])
fig.show(renderer="colab")

In [0]:
fig = px.line(confirmed_flow, x="Date", y="Confirmed",color='Country/Region',title="Timeline of confirmation of cases")
fig.update_traces(mode='lines+markers')
fig.show(renderer="colab")

In [0]:
fig = px.bar(confirmed_flow, x="Country/Region", y="Confirmed", color="Confirmed",animation_frame="Date", animation_group="Country/Region", range_y=[0,450000],title="The spread of CORONA?!",
             color_continuous_scale='reds',range_color=[1000,150000])
fig.show(renderer="colab")

#**Analysis of Deaths due to Corona Virus**

##**Building relevant dataframes**

In [0]:
top_10_death = death.sort_values(by=["4/7/20"],ascending=False,na_position='last')[0:10]
death_flow = top_10_death.melt(id_vars=["Province/State", "Country/Region", "Lat",'Long'],var_name="Date",value_name="Dead")


global_deaths = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Corona/total-daily-covid-deaths.csv")

for i,string in enumerate(global_deaths['Date']):
    x = datetime.strptime(string,'%b %d, %Y')
    global_deaths['Date'][i] = x.strftime('%Y-%m-%d')

global_deaths_world = global_deaths[global_deaths['Entity']=='World']
global_deaths_excluding_china = global_deaths[global_deaths['Entity']=='World excl. China']
deaths_china = global_deaths[global_deaths['Entity']=='China']

In [0]:
country_deaths = global_deaths.copy()
result = []
for code in country_deaths['Code']:
    a = pycountry.countries.get(alpha_3=code)
    if a is not None: a = code
    result.append(a)
country_deaths['Code'] = result
country_deaths = country_deaths.dropna()
con_10_total = country_deaths[country_deaths['Total confirmed deaths (deaths)']>=10]

In [0]:
global_deaths.head()

,Entity,Code,Date,Total confirmed deaths (deaths),Daily new confirmed deaths (deaths)
0,Afghanistan,AFG,2019-12-31,0,0
1,Afghanistan,AFG,2020-01-01,0,0
2,Afghanistan,AFG,2020-01-02,0,0
3,Afghanistan,AFG,2020-01-03,0,0
4,Afghanistan,AFG,2020-01-04,0,0


In [0]:
global_deaths_world.head()

,Entity,Code,Date,Total confirmed deaths (deaths),Daily new confirmed deaths (deaths)
10673,World,OWID_WRL,2019-12-31,0,0
10674,World,OWID_WRL,2020-01-01,0,0
10675,World,OWID_WRL,2020-01-02,0,0
10676,World,OWID_WRL,2020-01-03,0,0
10677,World,OWID_WRL,2020-01-04,0,0


In [0]:
global_deaths_excluding_china.head()

,Entity,Code,Date,Total confirmed deaths (deaths),Daily new confirmed deaths (deaths)
10773,World excl. China,NaN,2019-12-31,0,0
10774,World excl. China,NaN,2020-01-01,0,0
10775,World excl. China,NaN,2020-01-02,0,0
10776,World excl. China,NaN,2020-01-03,0,0
10777,World excl. China,NaN,2020-01-04,0,0


In [0]:
deaths_china.head()

,Entity,Code,Date,Total confirmed deaths (deaths),Daily new confirmed deaths (deaths)
2081,China,CHN,2019-12-31,0,0
2082,China,CHN,2020-01-01,0,0
2083,China,CHN,2020-01-02,0,0
2084,China,CHN,2020-01-03,0,0
2085,China,CHN,2020-01-04,0,0


In [0]:
con_10_total.head()

,Entity,Code,Date,Total confirmed deaths (deaths),Daily new confirmed deaths (deaths)
89,Afghanistan,AFG,2020-04-08,11,4
210,Albania,ALB,2020-03-29,10,1
211,Albania,ALB,2020-03-30,10,0
212,Albania,ALB,2020-03-31,12,2
213,Albania,ALB,2020-04-01,15,3


##**Plots**

In [0]:
fig = px.bar(top_10_death,x="Country/Region",y="4/7/20",color="4/7/20",title="Top 10 countries with most number of Death cases due to Corona Virus",
             color_continuous_scale='viridis',range_color=[1,20000])
fig.show(renderer="colab")

In [0]:
fig = px.line(death_flow, x="Date", y="Dead", color='Country/Region',title="History of Deaths")
fig.update_traces(mode='lines+markers')
fig.show(renderer="colab")

In [0]:
fig = px.bar(death_flow, x="Country/Region", y="Dead", color="Dead",animation_frame="Date", range_y=[0,20000],title="Deaths with time",
             color_continuous_scale='viridis',range_color=[1,20000])
fig.show(renderer="colab")

In [0]:
fig = go.Figure()
date = global_deaths_world['Date']
total_count_of_deaths = global_deaths_world['Total confirmed deaths (deaths)']
daily_count_of_deaths = global_deaths_world['Daily new confirmed deaths (deaths)']
# Create and style traces
fig.add_trace(go.Scatter(x=date, y=total_count_of_deaths, name='Total confirmed deaths',line=dict(color='firebrick')))
fig.add_trace(go.Scatter(x=date, y=daily_count_of_deaths, name = 'Daily new confirmed deaths',
                         line=dict(color='royalblue')))
# Edit the layout
fig.update_traces(mode='lines+markers')
fig.update_layout(title='Total Deaths worldwide due to corona analysis',
                   xaxis_title='Date',
                   yaxis_title='Deaths')


fig.show(renderer='colab')

In [0]:
fig = go.Figure()
date = global_deaths_world['Date']
y = global_deaths_excluding_china['Total confirmed deaths (deaths)']
z = deaths_china['Total confirmed deaths (deaths)']
# Create and style traces
fig.add_trace(go.Scatter(x=date, y=y, name='World excluding China',line=dict(color='firebrick')))
fig.add_trace(go.Scatter(x=date, y=z, name = 'China',
                         line=dict(color='royalblue')))
# Edit the layout
fig.update_traces(mode='lines+markers')
fig.update_layout(title='World V/S China- Total confirmed Deaths',
                   xaxis_title='Date',
                   yaxis_title='Deaths')


fig.show(renderer='colab')

In [0]:
fig = go.Figure()
date = global_deaths_world['Date']
y = global_deaths_excluding_china['Daily new confirmed deaths (deaths)']
z = deaths_china['Daily new confirmed deaths (deaths)']
# Create and style traces
fig.add_trace(go.Scatter(x=date, y=y, name='World excluding China',line=dict(color='firebrick')))
fig.add_trace(go.Scatter(x=date, y=z, name = 'China',
                         line=dict(color='royalblue')))
# Edit the layout
fig.update_traces(mode='lines+markers')
fig.update_layout(title='World V/S China - Daily deaths',
                   xaxis_title='Date',
                   yaxis_title='Deaths')


fig.show(renderer='colab')

In [0]:
fig = px.line(con_10_total,y="Total confirmed deaths (deaths)", color="Entity",
              width=1000,height=600,range_x=[5,40],range_y=[0,5000])
fig.update_traces(mode='lines+markers',)
fig.update_layout(title='Day wise analysis since the 10th confirmed death is respective countries',
                   xaxis_title='Day wise analysis of countries since their 10th confirmed death',
                   yaxis_title='Deaths')
fig.show(renderer='colab')

#**Analysis of recoveries over time**

In [0]:
top_10_recovered = recovered.sort_values(by=["4/7/20"],ascending=False,na_position='last')[0:10]

recovered_flow = top_10_recovered.melt(id_vars=["Province/State", "Country/Region", "Lat",'Long'],var_name="Date",value_name="Recovered")

##**Plots**

In [0]:
fig = px.bar(top_10_recovered,x="Country/Region",y="4/7/20",color="4/7/20",title="Top 10 Countries with the most number of recoveries")
fig.show(renderer="colab")

In [0]:
fig = px.line(recovered_flow, x="Date", y="Recovered", color='Country/Region',title="The road of recovery")
fig.update_traces(mode='lines+markers')
fig.show(renderer="colab")

In [0]:
fig = px.bar(recovered_flow, x="Country/Region", y="Recovered", color="Recovered",animation_frame="Date", range_y=[0,65000],title="Recovery through time")
fig.show(renderer="colab")

#**Confirmed - Recovered - Dead ?**

##**Building relevant frames**

In [0]:
c = (np.sum(confirmed.values[:,4:],axis=0))
r = (np.sum(recovered.values[:,4:],axis=0))
d = (np.sum(death.values[:,4:],axis=0))
days = [i for i in range(1,78)]
state=[]
for i in days:
    state.append('confirmed cases')
    state.append('recovered cases')
    state.append('death cases')
values=[]
for i in days:
    values.append(c[i-1])
    values.append(r[i-1])
    values.append(d[i-1])
day = []
for i in days:
    day.append(i)
    day.append(i)
    day.append(i)

global_pandemic = pd.DataFrame({'Day':day,'State':state,'Count':values})

In [0]:
fig = px.bar(global_pandemic, x="Count", y="State", color="Count",animation_frame="Day", range_x=[0,1500000],title="Again a worldwide analysis",orientation='h',
             color_continuous_scale='solar',range_color=[10000,600000])
fig.show(renderer="colab")

#**Median Age of countries V/S Death-to-Confirmed Ratio of cases** 

##**Building relevant data frames**

In [0]:
median_age = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Corona/data.csv")
median_age.head()

,Place,Median,Median Male,Median Female
0,Monaco,53.1,51.7,54.5
1,Japan,47.3,46.0,48.7
2,Germany,47.1,46.0,48.2
3,Saint Pierre and Miquelon,46.5,46.0,47.0
4,Italy,45.5,44.4,46.5


In [0]:
df = global_deaths.copy(deep=True)
df = df[df['Date']=='2020-04-08']
result=[]
for name in df['Entity']:
    a = pycountry.countries.get(name=name)
    if a is not None: a = a.alpha_3
    result.append(a)

df['Code'] = result
df = df.dropna()

In [0]:

result=[]
for name in median_age['Place']:
    a = pycountry.countries.get(name=name)
    if a is not None: a = a.alpha_3
    result.append(a)

median_age['CODE'] = result

In [0]:
result = []
for code in df['Code']:
    result.append(np.sum(np.asarray(median_age['Median'])*np.asarray(median_age['CODE']==code)))
df['median']=result

In [0]:
df = df.sort_values(by=['Total confirmed deaths (deaths)'])
df = df[-30:]

In [0]:
top_30_confirmed = confirmed.sort_values(by=["4/7/20"],ascending=True,na_position='last')[-30:]

In [0]:
x = df[['Entity','Total confirmed deaths (deaths)','median']].copy()
y = top_30_confirmed[['Country/Region','4/7/20']].copy()
array = y.values
array[-1][0]='United States'
y['Country/Region'] = list(array[:,0])
new_df = x.merge(y,how='inner',left_on='Entity',right_on ="Country/Region")

In [0]:
array = new_df.values
dead_by_confirm = np.divide(array[:,1],array[:,4])
new_df['dead/confirm ratio'] = list(dead_by_confirm)
new_df = new_df.sort_values(by=['dead/confirm ratio'])
new_df = new_df[new_df['4/7/20']!=9340]
new_df.head()

,Entity,Total confirmed deaths (deaths),median,Country/Region,4/7/20,dead/confirm ratio
13,Germany,1861,47.1,Germany,107663,0.017285
5,Austria,243,44.0,Austria,12639,0.019226
12,Turkey,725,30.9,Turkey,34109,0.021255
0,Poland,129,40.7,Poland,4848,0.026609
6,Portugal,345,42.2,Portugal,12442,0.027729


##**Plots**

In [0]:
fig = px.bar(new_df, x='Entity', y='dead/confirm ratio',
             hover_data=['dead/confirm ratio', 'Total confirmed deaths (deaths)','4/7/20','median'], color='median', height=400,title='dead/confrim ratio V/S median bar chart')
fig.show(renderer='colab')

In [0]:
fig = px.scatter(new_df, x="median", y="dead/confirm ratio", color="dead/confirm ratio", hover_data=['Entity','median','dead/confirm ratio'],
                 color_continuous_scale='viridis')
fig.update_traces(mode='markers', marker_size=40)
fig.update_layout(title='Better visualisation of "dead/confirm ratio V/S median"',
                  yaxis_zeroline=False, xaxis_zeroline=False)

fig.show(renderer='colab')

#**Conclusion**

The Corona Virus is going to be the cause of recession in the world. Almost entire world is at hault today. Within a span of approximately 5 months, globally 1.5 Million people have got the virus, and approximately 100 thousand people have died. Countries like Italy, Spain and USA seem to have suffered the most. No signs of hope are visible. People are losing their families, friends and loved ones. Countries are falling short of graveyards to bury. 

The analysis also leads to many questions as to how China didn't get affected by the virus to a huge extent despite being the native ground to the virus. How come entire world is under suffering but China has got over it? Germany is among the countries with the most number of confirmed cases yet the mortality rate is low there. What brings the country to this safer side, despite being surrounded by the other European Countries struggling hard to get through. Data analysis and research say that Corona Virus can only be controlled by increasing the amount of testings. Because a person witnesses symptoms of the virus approximately 15-20 days after getting exposed to the virus. Hence for those 15-20 days, the person becomes a carrier of the virus. If we are able to do testings at a faster rate, we can identify carriers, well before they start showing symptoms and hence prevent the widespread. 

This is our duty as data analyists and Machine Learning practioners to find patterns among the data and try to make predictions and solutions. 

#**WE ALL ARE TOGETHER IN THIS FIGHT. STAY HOME ! STAY SAFE !**